In [1]:
import json
import os
from PIL import Image, ImageOps, ImageFilter

# Rutas de archivos y carpetas
IMAGES_PATH = os.path.abspath('../mnt/Dataset/images/')
ANNOTATIONS_FILE = os.path.abspath('../mnt/Dataset/annotations/annotations.json')
OUTPUT_ANNOTATIONS_DIR = os.path.abspath('../mnt/V3/annotations')
OUTPUT_IMAGES_DIR = os.path.abspath('../mnt/V3/images')


In [2]:
def create_directories():
    """
    Crea los directorios de salida si no existen.
    """
    os.makedirs(OUTPUT_ANNOTATIONS_DIR, exist_ok=True)
    os.makedirs(OUTPUT_IMAGES_DIR, exist_ok=True)

In [3]:
def load_annotations(file_path):
    """
    Carga las anotaciones desde un archivo JSON.
    """
    with open(file_path, 'r') as f:
        return json.load(f)

In [4]:
def normalize_pose(pose, image_width, image_height):
    """
    Normaliza las coordenadas de las poses con respecto a las dimensiones de la imagen.
    """
    return [
        [x / image_width, y / image_height, c]
        for x, y, c in pose
    ]

In [ ]:
def preprocess_data(annotations):
    """
    Preprocesa las anotaciones, aplica normalización, filtros de imagen y redimensionamiento.
    """
    data = []
    for annotation in annotations:
        # Ignorar anotaciones sin pose1 o pose2
        if 'pose1' not in annotation or 'pose2' not in annotation:
            continue

        image_name = annotation['image'] + '.jpg'
        image_path = os.path.join(IMAGES_PATH, image_name)

        # Verificar que la imagen exista
        if not os.path.exists(image_path):
            continue

        # Procesar la imagen
        image = Image.open(image_path)
        width, height = image.size

        # Normalizar poses
        pose1 = normalize_pose(annotation.get('pose1'), width, height)
        pose2 = normalize_pose(annotation.get('pose2'), width, height)

        # Redimensionar la imagen
        resized_image = image.resize((256, 256))

        # Convertir a blanco y negro
        bw_image = ImageOps.grayscale(resized_image).convert("RGB")

        # Aplicar filtro Sobel para detección de bordes
        sobel_image = bw_image.filter(ImageFilter.FIND_EDGES)

        # Agregar datos preprocesados
        data.append({
            'image': sobel_image,
            'pose1': pose1,
            'pose2': pose2,
            'position': annotation['position'],
            'frame': annotation.get('frame') 
        })

    return data

In [6]:
def save_images(data, output_dir):
    """
    Guarda las imágenes procesadas en el directorio de salida.
    """
    for idx, item in enumerate(data):
        file_name = f'{str(idx+1).zfill(7)}.jpg'
        item['image'].save(os.path.join(output_dir, file_name))

In [ ]:
def save_annotations(data, output_file):
    """
    Guarda las anotaciones en un archivo JSON en el directorio de salida.
    """
    annotations = [
        {
            'Image': f'{str(idx+1).zfill(7)}',
            'Pose1': item['pose1'],
            'Pose2': item['pose2'],
            'Position': item['position'],
            'Frame': item['frame']  
        }
        for idx, item in enumerate(data)
    ]
    with open(output_file, 'w') as f:
        json.dump(annotations, f)


In [8]:
def main():
    """
    Función principal que ejecuta el flujo completo de preprocesamiento.
    """
    # Crear directorios de salida
    create_directories()

    # Cargar anotaciones
    annotations = load_annotations(ANNOTATIONS_FILE)

    # Preprocesar datos
    data = preprocess_data(annotations)

    # Guardar imágenes y anotaciones preprocesadas
    save_images(data, OUTPUT_IMAGES_DIR)
    save_annotations(data, os.path.join(OUTPUT_ANNOTATIONS_DIR, 'annotations_preprocessed.json'))

    print(f'Proceso completado. Se han guardado {len(data)} imágenes en {OUTPUT_IMAGES_DIR} y las anotaciones en {OUTPUT_ANNOTATIONS_DIR}.')

if __name__ == '__main__':
    main()


OSError: [Errno 22] Invalid argument